## Домашнее задание №4

Попробуйте улучшить точность распознавания образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложите анализ с описанием того, что улучшает работу нейронной сети, а что ухудшает

Опишите в анализе, какие изменения необходимо было бы внести в получившуюся нейронную сеть, если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET

In [ ]:
from __future__ import print_function
#import keras
# Я поменяла импорт, т.к. keras устанавливается вместе с tensorflow как надстройка над ним и через такой импорт оптимизатор RMSprop становится доступен
# RMSprop как добавляет в градиентный спуск инерцию, так и меняет размер градиентного шага:  https://keras.io/api/optimizers/rmsprop/
from keras.utils import np_utils
from tensorflow import keras 
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

# установка параметров нейросети
batch_size = 64
num_classes = 10
epochs = 6
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# инициализация RMSprop optimizer
opt = keras.optimizers.Adam(lr=0.0001, decay=1e-6)

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation в реальном времени')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    # запуск data augmentation через fit
    #datagen.fit(x_train)

    # запуск data augmentation через fit_generator
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# сохранение модели и весов
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('сохранить обученную модель как %s ' % model_path)

# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Использование data augmentation в реальном времени
Epoch 1/6


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:113: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


782/782 [==============================] - 264s 337ms/step - loss: 1.9085 - accuracy: 0.3006 - val_loss: 1.6063 - val_accuracy: 0.4253
Epoch 2/6
782/782 [==============================] - 260s 332ms/step - loss: 1.6197 - accuracy: 0.4068 - val_loss: 1.4448 - val_accuracy: 0.4809
Epoch 3/6
782/782 [==============================] - 258s 329ms/step - loss: 1.5070 - accuracy: 0.4489 - val_loss: 1.3595 - val_accuracy: 0.5093
Epoch 4/6
782/782 [==============================] - 261s 333ms/step - loss: 1.4371 - accuracy: 0.4829 - val_loss: 1.3063 - val_accuracy: 0.5305
Epoch 5/6
782/782 [==============================] - 259s 331ms/step - loss: 1.3888 - accuracy: 0.4995 - val_loss: 1.2379 - val_accuracy: 0.5557
Epoch 6/6
782/782 [==============================] - 258s 329ms/step - loss: 1.3383 - accuracy: 0.5210 - val_loss: 1.1803 - val_accuracy: 0.5814
сохранить обученную модель как /content/saved_models/keras_cifar10_trained_model.h5 
313/313 [==============================] - 12s 37ms/ste

Увиличив кол-во эпох, чуть чуть увеличив число бачей и заменив оптимизатор на адам accuracy выросло. Так же были произведены тесты с большим числом бачейи заменой функции активаации на tanh что уменьшило accuracy.

Для работы с MNIST, CIFAR100 и IMAGENET пришлось бы изменить num_classes, разделение тренировочной и тестовой выборки